<a href="https://colab.research.google.com/github/Tanny-H/dataviz2025/blob/main/6630202808_midterm2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
crime_scene_report=pd.read_csv('/content/drive/MyDrive/midterm dataviz/crime_scene_report.csv')

In [9]:
csr = crime_scene_report
csr.head()

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,Pandas City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,Pandas City


In [12]:
filtered_crime_scene_report

,date,type,description,city
1227,20180115,murder,Security footage shows that there were 2 witne...,Pandas City


In [11]:
filtered_crime_scene_report

,description
1227,Security footage shows that there were 2 witne...


##description :

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".

ขั้นตอนต่อไปเป็นการหาพยาน

คนแรกคือ ผู้ที่อยู่บ้านเลขที่ หลังสุดท้าย บนถนน Northwestern Dr

คนที่สองคือ Annabel ที่อยู่บน Franklin Ave

In [16]:
person.head()

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684


In [21]:
# ----------------------------
# 1) หาพยานคนแรก (บ้านเลขที่มากที่สุดบน Northwestern Dr)
northwestern = person[person['address_street_name'] == "Northwestern Dr"]
witness1 = northwestern.loc[northwestern['address_number'].idxmax()]

# ----------------------------
# 2) หาพยานคนที่สอง (ชื่อ Annabel ที่อยู่บน Franklin Ave)
witness2 = person[
    (person['name'].str.contains("Annabel", case=False, na=False)) &
    (person['address_street_name'].str.contains("Franklin Ave", case=False, na=False))
]

print("พยานคนแรก:")
print(witness1)

print("\nพยานคนที่สอง:")
print(witness2)

พยานคนแรก:
id                               14887
name                    Morty Schapiro
license_id                      118009
address_number                    4919
address_street_name    Northwestern Dr
ssn                          111564949
Name: 499, dtype: object

พยานคนที่สอง:
        id            name  license_id  address_number address_street_name  \
665  16371  Annabel Miller      490173             103        Franklin Ave   

           ssn  
665  318771143  


##ตรวจสอบคำให้การของทั้งสองคน

In [23]:
# Load the interview.csv file
interview = pd.read_csv("/content/drive/MyDrive/midterm dataviz/interview.csv")

# Merge interview data with person data to get names
interview_with_names = pd.merge(interview, person, left_on='person_id', right_on='id')

# Find the interviews for Morty Schapiro and Annabel Miller
witness_names = ["Morty Schapiro", "Annabel Miller"]
witness_interviews = interview_with_names[interview_with_names['name'].isin(witness_names)]

print(witness_interviews)

      person_id                                         transcript     id  \
4988      14887  I heard a gunshot and then saw a man run out. ...  14887   
4989      16371  I saw the murder happen, and I recognized the ...  16371   

                name  license_id  address_number address_street_name  \
4988  Morty Schapiro      118009            4919     Northwestern Dr   
4989  Annabel Miller      490173             103        Franklin Ave   

            ssn  
4988  111564949  
4989  318771143  


## คำให้การ

Morty Schapiro:
“I heard a gunshot and then saw a man run out. He had a Get Fit Now Gym bag. The membership number on the bag started with ‘48Z’. Only gold members have those bags. The man got into a car with a plate that included ‘H42W’.”

Annabel Miller:
“I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.”

In [24]:

# โหลดข้อมูล
members = pd.read_csv("/content/drive/MyDrive/midterm dataviz/get_fit_now_member.csv")
checkins = pd.read_csv("/content/drive/MyDrive/midterm dataviz/get_fit_now_check_in.csv")
licenses = pd.read_csv("/content/drive/MyDrive/midterm dataviz/drivers_license.csv")


# -------------------------------
# 1) คัดเลือกสมาชิกที่เริ่มด้วย "48Z" และเป็น gold member
suspect_members = members[
    (members['id'].str.startswith("48Z")) &
    (members['membership_status'] == "gold")
]

# -------------------------------
# 2) ตรวจสอบว่าใครเช็กอินในวันที่ 2018-01-09
suspect_checkins = checkins[
    (checkins['membership_id'].isin(suspect_members['id'])) &
    (checkins['check_in_date'] == "2018-01-09")
]

# -------------------------------
# 3) หาคนเหล่านี้จาก person.csv (เชื่อมด้วย person_id)
suspect_people = person[person['id'].isin(suspect_members['person_id'])]

# -------------------------------
# 4) cross-check กับทะเบียนรถ (ต้องมี H42W ใน license plate)
final_suspects = suspect_people.merge(licenses, left_on="license_id", right_on="id")
final_suspects = final_suspects[final_suspects['plate_number'].str.contains("H42W", na=False)]

print("ฆาตกรที่เป็นไปได้:")
print(final_suspects[['name', 'plate_number']])


ฆาตกรที่เป็นไปได้:
            name plate_number
0  Jeremy Bowers       0H42W2


## ฆาตกรที่เป็นไปได้คือ Jeremy Bowers

## Jeremy's transcript
I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.


In [26]:
# Find Jeremy Bowers' interview
jeremy_interview = interview_with_names[interview_with_names['name'] == 'Jeremy Bowers']
display(jeremy_interview)

,person_id,transcript,id,name,license_id,address_number,address_street_name,ssn
4990,67318,I was hired by a woman with a lot of money. I ...,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


## ขั้นตอนต่อไปเป็นการหาตัวผู้บงการ โดยใช้คำสารภาพจาก Jeremy

In [28]:
fb_checkin = pd.read_csv("/content/drive/MyDrive/midterm dataviz/facebook_event_checkin.csv")

In [34]:
# --- ขั้นตอน 1: กรองจาก drivers_license.csv ---
candidates = licenses[
    (licenses["gender"] == "female") &
    (licenses["hair_color"] == "red") &
    (licenses["height"].between(65, 67)) &
    (licenses["car_make"] == "Tesla") &
    (licenses["car_model"] == "Model S")
]

# --- ขั้นตอน 2: join กับ person.csv เพื่อหาชื่อ ---
candidates = candidates.merge(person, left_on="id", right_on="license_id")

# --- ขั้นตอน 3: ตรวจสอบการเข้าร่วม SQL Symphony Concert ---
# Convert 'date' column to string before using .str accessor
fb_checkin['date'] = fb_checkin['date'].astype(str)

concert_checkin = fb_checkin[
    (fb_checkin["event_name"] == "SQL Symphony Concert") &
    (fb_checkin["date"].str.startswith("201712")) # Corrected to match integer date format
]

# นับจำนวนครั้งที่ไป concert
concert_count = concert_checkin.groupby("person_id").size().reset_index(name="count")

# --- ขั้นตอน 4: รวมข้อมูลเข้าด้วยกัน ---
final = candidates.merge(concert_count, left_on="id_y", right_on="person_id")

# --- ขั้นตอน 5: เลือกเฉพาะคนที่ไป concert 3 ครั้ง ---
final = final[final["count"] == 3]

# แสดงผลชื่อผู้บงการ
print("ผู้บงการ:")
print(final[["name", "height", "hair_color", "car_make", "car_model", "count"]])

ผู้บงการ:
               name  height hair_color car_make car_model  count
0  Miranda Priestly      66        red    Tesla   Model S      3


## พบว่าผู้บงการคือ Miranda Priestly